# Stock Price Prediction using ML model
In this session, we'll learn how to build a ML model for predicting **%change of stock prices of the next day** of stocks in SET index (Stock Exchange of Thailand). Thus, we should be able to use the prediction to buy stocks that are going up the next day, make profits, and hopefully get rich!

This session is divided into the following 5 notebooks.
1. `1_collect_data.ipynb`
2. `2_eda.ipynb`
3. `3_features_prep.ipynb` (current notebook)
4. `4_make_prediction.ipynb`
5. `5_evaluation.ipynb`

# Feature Preparation
In this notebook, we will prepare features from the data we have collected and explored from the previous step. Preparing features is a really important step in extracting useful information and context for a ML model to learn on and make a right prediction. This is particularly true for tabular data where rich features outweighs complex models. Even though, it is one of the most time consuming steps in building ML model, it worths the effort as good and insightful features could potentially boost your model performance!

In [55]:
import datetime as dt
from dateutil.relativedelta import relativedelta

from glob import glob
from pathlib import Path
import os
import json

import numpy as np
import pandas as pd
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm.notebook import tqdm

import talib
from talib import RSI, BBANDS, MACD, ATR

In [56]:
sns.set_style('darkgrid')
idx = pd.IndexSlice

# Price features
We'll extract some popular indicators for predicting stock price from the price data. The features are far from complete as there are plenty more of indicators you can add to the model. You can take a look at an extensive list of indicators at the `TA-Lib` documentary page [here](https://github.com/TA-Lib/ta-lib-python?tab=readme-ov-file#supported-indicators-and-functions).

In [57]:
MONTH = 20
YEAR = 12 * MONTH

In [58]:
price_df = pd.read_csv("data/set_price.csv", parse_dates=["date"])

In [59]:
price_df.head()

,symbol,date,open,high,low,close,volume,dividends,stock splits,capital gains
0,24CS,2022-10-03,7.10,10.20,7.10,10.20,559465900,0.0,0.0,NaN
1,24CS,2022-10-04,10.70,11.10,7.15,7.15,330707400,0.0,0.0,NaN
2,24CS,2022-10-05,5.85,6.45,5.05,5.15,361028900,0.0,0.0,NaN
3,24CS,2022-10-06,5.40,5.45,4.70,5.20,232679200,0.0,0.0,NaN
4,24CS,2022-10-07,5.10,5.15,4.76,5.00,131778400,0.0,0.0,NaN


In [60]:
# filter abnormal close price
price_df = price_df[price_df["close"] > 0]

# drop unused columns
price_df = price_df.drop(columns=["capital gains"])

In [61]:
# set index
price_df = price_df.set_index(["symbol","date"]).sort_index()

In [62]:
# remove stock with insufficient data
min_obs = 2 * YEAR
nobs = price_df.groupby('symbol').size()
keep = nobs[nobs > min_obs].index

price_df = price_df.loc[idx[keep, :], :]

## 21-day moving average of trading value
We extract this feature to measure how hot or trendy a stock is over last 21 days period.

In [63]:
# calculate daily trading value
price_df["value"] = price_df["close"] * price_df["volume"]

In [64]:
price_df["value"] = price_df["value"].div(1e6)

val_ma = price_df["value"]\
    .unstack("symbol")\
    .rolling(window=21, min_periods=1)\
    .mean()

price_df["val_rank"] = val_ma\
                        .rank(axis=1, ascending=False)\
                        .stack("symbol")\
                        .swaplevel()

## Relative Strength Index (RSI)
RSI is usually used to determined whether a stock is overly bought or sold already, so it might be a good time to execute a trade.

In [65]:
price_df["rsi"] = price_df.groupby(level="symbol", group_keys=False).close.apply(RSI)

## Bollinger bands
This indicator is used for measuring volatility of a stock and also determining whether a stock is undervalue or overvalue.

![BB](img/BB.png)

In [66]:
def compute_bb(close):
    high, mid, low = BBANDS(close, timeperiod=20)
    return pd.DataFrame({'bb_high': high, 'bb_low': low}, index=close.index)

In [67]:
price_df = (price_df.join(price_df
                      .groupby(level='symbol', group_keys=False)
                      .close
                      .apply(compute_bb)))

In [68]:
price_df['bb_high'] = price_df.bb_high.sub(price_df.close).div(price_df.bb_high).apply(np.log1p)
price_df['bb_low'] = price_df.close.sub(price_df.bb_low).div(price_df.close).apply(np.log1p)

## Average True Range (ATR)
The indicator mesures level of volatility of a stock for the period evaluated.

In [69]:
price_df['NATR'] = price_df.groupby(level='symbol', 
                                group_keys=False).apply(lambda x: 
                                                        talib.NATR(x.high, x.low, x.close))

In [70]:
def compute_atr(stock_data):
    df = ATR(stock_data.high, stock_data.low, 
             stock_data.close, timeperiod=14)
    return df.sub(df.mean()).div(df.std())

In [71]:
price_df['ATR'] = (price_df.groupby('symbol', group_keys=False)
                 .apply(compute_atr))

## Moving Average Convergence/Divergence (MACD)
MACD is one of the most popular indicators used for indentifying price trends and measuring trend momentum.

In [1]:
# price_df["PPO"] = price_df.groupby(level="symbol", group_keys=False)["close"].apply(talib.PPO)

In [73]:
def compute_macd(close):
    macd = MACD(close)[0]
    return (macd - np.mean(macd)) / np.std(macd)

In [74]:
price_df["MACD"] = (price_df.
                    groupby("symbol", group_keys=False)
                    .close
                    .apply(compute_macd)
                    )

In [75]:
price_df.head()

open   high   low  close     volume  dividends  \
symbol date                                                          
24CS   2022-10-03   7.10  10.20  7.10  10.20  559465900        0.0   
       2022-10-04  10.70  11.10  7.15   7.15  330707400        0.0   
       2022-10-05   5.85   6.45  5.05   5.15  361028900        0.0   
       2022-10-06   5.40   5.45  4.70   5.20  232679200        0.0   
       2022-10-07   5.10   5.15  4.76   5.00  131778400        0.0   

                   stock splits        value  val_rank  rsi  bb_high  bb_low  \
symbol date                                                                    
24CS   2022-10-03           0.0  5706.552073       1.0  NaN      NaN     NaN   
       2022-10-04           0.0  2364.557942       1.0  NaN      NaN     NaN   
       2022-10-05           0.0  1859.298869       1.0  NaN      NaN     NaN   
       2022-10-06           0.0  1209.931796       2.0  NaN      NaN     NaN   
       2022-10-07           0.0   658.892000       2.0  NaN      NaN     NaN   

                   NATR  ATR  PPO  MACD  
symbol date                              
24CS   2022-10-03   NaN  NaN  NaN   NaN  
       2022-10-04   NaN  NaN  NaN   NaN  
       2022-10-05   NaN  NaN  NaN   NaN  
       2022-10-06   NaN  NaN  NaN   NaN  
       2022-10-07   NaN  NaN  NaN   NaN

# Join company information

In [76]:
company_info_df = pd.read_csv("data/set_company_info.csv")

In [77]:
company_info_df.head()

,symbol,industry,sector
0,24CS,Building Products & Equipment,Industrials
1,2S,Steel,Basic Materials
2,3BBIF,NaN,NaN
3,A,Real Estate - Development,Real Estate
4,A5,Real Estate - Development,Real Estate


In [78]:
company_info_df.set_index("symbol", inplace=True)

In [79]:
data_df = price_df.join(company_info_df, on="symbol")

In [80]:
data_df.head()

open   high   low  close     volume  dividends  \
symbol date                                                          
24CS   2022-10-03   7.10  10.20  7.10  10.20  559465900        0.0   
       2022-10-04  10.70  11.10  7.15   7.15  330707400        0.0   
       2022-10-05   5.85   6.45  5.05   5.15  361028900        0.0   
       2022-10-06   5.40   5.45  4.70   5.20  232679200        0.0   
       2022-10-07   5.10   5.15  4.76   5.00  131778400        0.0   

                   stock splits        value  val_rank  rsi  bb_high  bb_low  \
symbol date                                                                    
24CS   2022-10-03           0.0  5706.552073       1.0  NaN      NaN     NaN   
       2022-10-04           0.0  2364.557942       1.0  NaN      NaN     NaN   
       2022-10-05           0.0  1859.298869       1.0  NaN      NaN     NaN   
       2022-10-06           0.0  1209.931796       2.0  NaN      NaN     NaN   
       2022-10-07           0.0   658.892000       2.0  NaN      NaN     NaN   

                   NATR  ATR  PPO  MACD                       industry  \
symbol date                                                              
24CS   2022-10-03   NaN  NaN  NaN   NaN  Building Products & Equipment   
       2022-10-04   NaN  NaN  NaN   NaN  Building Products & Equipment   
       2022-10-05   NaN  NaN  NaN   NaN  Building Products & Equipment   
       2022-10-06   NaN  NaN  NaN   NaN  Building Products & Equipment   
       2022-10-07   NaN  NaN  NaN   NaN  Building Products & Equipment   

                        sector  
symbol date                     
24CS   2022-10-03  Industrials  
       2022-10-04  Industrials  
       2022-10-05  Industrials  
       2022-10-06  Industrials  
       2022-10-07  Industrials

# Compute return

## Historical returns
Returns and return ranking over different periods could also be extracted as features.

In [81]:
# Historical returns
T = [1, 5, 10, 21, 42, 63]
by_sym = data_df.groupby(level="symbol")["close"]
for t in T:
    data_df[f"r{t:02}"] = by_sym.pct_change(t)

In [82]:
# Daily historical return deciles (return ranking)
for t in T:
    data_df[f"r{t:02}dec"] = (data_df[f"r{t:02}"].dropna()
                              .groupby(level="date", group_keys=False)
                              .apply(lambda x: pd.qcut(x,
                                                       q=10,
                                                       labels=False,
                                                       duplicates="drop")))

In [83]:
# Daily sector return deciles (return ranking in a sector)
for t in T:
    data_df[f"r{t:02}q_sector"] = (data_df
                                   .groupby(["date","sector"])[f"r{t:02}"]
                                   .transform(lambda x: pd.qcut(x,
                                                                 q=5,
                                                                labels=False,
                                                                duplicates="drop")))

## Forward returns
Here we'll create multiple forward returns over different period, but for our application, we'll only use the forward return over the next day.

In [84]:
for t in [1, 5, 21]:
    data_df[f'r{t:02}_fwd'] = data_df.groupby(level='symbol')[f'r{t:02}'].shift(-t)

# Join income statement
In this step, we'll join the annual income statement data to our feature. Here, we'll join 3 previous annual income statement corresponding to date of each row of our data. What we need to be really careful when joining data with longer update frequency (obviously price data is updated daily, but annual income statement is updated annually!) is to make sure about when it is available. Thus, we would like to avoid joining the income statement to the price data where the income statement is not actually available.

In [85]:
incm_stmt_df = pd.read_csv("data/set_incm_stmt.csv")

In [86]:
# parse date column
incm_stmt_df["date"] = pd.to_datetime(incm_stmt_df["date"])
incm_stmt_df["realized_dt"] = incm_stmt_df["date"] + pd.DateOffset(days=100)

In [87]:
# drop unused and mostly empty columns
unused_cols_income = ["date", ]
incm_stmt_df.drop(columns=unused_cols_income, inplace=True)

In [88]:
MAX_PCT_NULL = .95

income_null = incm_stmt_df.isna().mean()
drop_cols_income = income_null[income_null >= MAX_PCT_NULL].index
drop_cols_income

Index(['Otherunder Preferred Stock Dividend', 'Other Taxes',
       'Rent And Landing Fees', 'Restructuring And Mergern Acquisition',
       'Average Dilution Earnings', 'Amortization',
       'Amortization Of Intangibles Income Statement',
       'Depreciation Income Statement', 'Insurance And Claims',
       'Preferred Stock Dividends', 'Net Income From Tax Loss Carryforward',
       'Research And Development', 'Earnings From Equity Interest Net Of Tax',
       'Net Income Extraordinary', 'Excise Taxes'],
      dtype='object')

In [89]:
incm_stmt_df.drop(columns=drop_cols_income, inplace=True)

In [90]:
# rearrange columns
incm_stmt_df = incm_stmt_df[["symbol", "realized_dt"] + [col for col in incm_stmt_df.columns if col not in ["symbol", "realized_dt"]]]

In [91]:
incm_stmt_df.head()

,symbol,realized_dt,Tax Effect Of Unusual Items,Tax Rate For Calcs,Normalized EBITDA,Total Unusual Items,Total Unusual Items Excluding Goodwill,Net Income From Continuing Operation Net Minority Interest,Reconciled Depreciation,Reconciled Cost Of Revenue,EBITDA,EBIT,Net Interest Income,Interest Expense,Interest Income,Normalized Income,Net Income From Continuing And Discontinued Operation,Total Expenses,Diluted Average Shares,Basic Average Shares,Diluted EPS,Basic EPS,Diluted NI Availto Com Stockholders,Net Income Common Stockholders,Net Income,Net Income Including Noncontrolling Interests,Net Income Continuous Operations,Tax Provision,Pretax Income,Other Income Expense,Other Non Operating Income Expenses,Special Income Charges,Gain On Sale Of Ppe,Other Special Charges,Gain On Sale Of Security,Net Non Operating Interest Income Expense,Interest Expense Non Operating,Interest Income Non Operating,Operating Income,Operating Expense,Selling General And Administration,Selling And Marketing Expense,General And Administrative Expense,Other Gand A,Salaries And Wages,Gross Profit,Cost Of Revenue,Total Revenue,Operating Revenue,Minority Interests,Impairment Of Capital Assets,Total Operating Income As Reported,Total Other Finance Cost,Other Operating Expenses,Depreciation Amortization Depletion Income Statement,Depreciation And Amortization In Income Statement,Earnings From Equity Interest,Provision For Doubtful Accounts,Rent Expense Supplemental,Write Off,Net Income Discontinuous Operations,Gain On Sale Of Business
0,24CS,2024-04-09,2.478119e+05,0.195597,-44136736.0,1266954.0,1266954.0,-45071044.0,9617938.0,6.714324e+08,-42869782.0,-52487720.0,-2439536.0,3542679.0,1103143.0,-4.609019e+07,-45071044.0,7.427603e+08,430000000.0,430000000.0,-0.100000,-0.100000,-45071044.0,-45071044.0,-45071044.0,-45071044.0,-45071044.0,-10959355.0,-56030399.0,7674673.0,6407719.0,1266954.0,1214952.0,-52002.0,NaN,-2439536.0,3542679.0,1103143.0,-61265536.0,71327944.0,71327944.0,9318398.0,62009546.0,62009546.0,NaN,10062408.0,6.714324e+08,6.814948e+08,6.814948e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,24CS,2023-04-10,5.479578e+05,0.223307,41876591.0,2453829.0,2453829.0,24494231.0,7726356.0,8.700639e+08,44330420.0,36604064.0,-4082461.0,5067487.0,985026.0,2.258836e+07,24494231.0,9.456595e+08,325513699.0,325513699.0,0.080000,0.080000,24494231.0,24494231.0,24494231.0,24494231.0,24494231.0,7042346.0,31536577.0,2710901.0,257072.0,1295046.0,1295046.0,NaN,1158783.0,-4082461.0,5067487.0,985026.0,32908137.0,75595530.0,75595530.0,NaN,NaN,NaN,NaN,108503667.0,8.700639e+08,9.785676e+08,9.785676e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,24CS,2022-04-10,0.000000e+00,0.241495,35101165.0,0.0,0.0,19455578.0,6959244.0,5.637045e+08,35101165.0,28141921.0,-2013217.0,2492033.0,478816.0,1.945558e+07,19455578.0,6.139717e+08,430000000.0,430000000.0,0.045246,0.045246,19455578.0,19455578.0,19455578.0,19455578.0,19455578.0,6194310.0,25649888.0,161675.0,161675.0,0.0,0.0,NaN,NaN,-2013217.0,2492033.0,478816.0,27501430.0,50267186.0,50267186.0,19250000.0,31020000.0,2560000.0,28460000.0,77768616.0,5.637045e+08,6.414731e+08,6.414731e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,24CS,2021-04-10,0.000000e+00,0.352007,18310000.0,NaN,NaN,6940000.0,6310000.0,3.545600e+08,18310000.0,12000000.0,-1290000.0,1290000.0,NaN,6.940000e+06,6940000.0,3.905600e+08,430000000.0,430000000.0,0.016140,0.016140,6940000.0,6940000.0,6940000.0,6940000.0,6940000.0,3770000.0,10710000.0,720000.0,720000.0,NaN,NaN,NaN,NaN,-1290000.0,1290000.0,NaN,10880000.0,36000000.0,36000000.0,17830000.0,18170000.0,2100000.0,16070000.0,46880000.0,3.545600e+08,4.014400e+08,4.014400e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2S,2024-04-09,-1.027415e+06,0.037799,248561000.0,-27181000.0,-27181000.0,160083000.0,55027000.0,6.351661e+09,221380000.0,166353000.0,3354000.0,264000.0,3618000.0,1.862366e+08,160083000.0,6.625824e+09,549996000.0,549996000.0,0.290000,0.290000,160083000.0,160083000.0,160

In [92]:
data_df.reset_index(inplace=True)

In [93]:
symbol_col = "symbol"
date_col_feat = "date"
date_col_income = "realized_dt"

map_df = data_df[[symbol_col, date_col_feat]].merge(incm_stmt_df[[symbol_col, date_col_income]], on=symbol_col)
print(map_df.shape)

map_df = map_df.query("realized_dt < date")

map_df["realized_dt_rank"] = map_df.groupby(["symbol", "date"])["realized_dt"].rank(ascending=False)

(3734661, 3)


In [94]:
map_df.head()

,symbol,date,realized_dt,realized_dt_rank
2,24CS,2022-10-03,2022-04-10,1.0
3,24CS,2022-10-03,2021-04-10,2.0
6,24CS,2022-10-04,2022-04-10,1.0
7,24CS,2022-10-04,2021-04-10,2.0
10,24CS,2022-10-05,2022-04-10,1.0


In [95]:
N_YEAR_LOOKBACK = 3

map_df = map_df.query(f"realized_dt_rank <= {N_YEAR_LOOKBACK}")
map_df = map_df.set_index(["symbol", "date", "realized_dt_rank"]).unstack(-1)

# flatten columns
map_df.columns = ["_".join(map(lambda x: str(x), col)).replace(".0", "") for col in map_df.columns]

In [96]:
map_df.head()

realized_dt_1 realized_dt_2 realized_dt_3
symbol date                                                
24CS   2022-10-03    2022-04-10    2021-04-10           NaT
       2022-10-04    2022-04-10    2021-04-10           NaT
       2022-10-05    2022-04-10    2021-04-10           NaT
       2022-10-06    2022-04-10    2021-04-10           NaT
       2022-10-07    2022-04-10    2021-04-10           NaT

In [97]:
data_df = data_df.set_index(["symbol", "date"])
data_df = data_df.join(map_df, how="left")

In [98]:
data_df[["realized_dt_1", "realized_dt_2", "realized_dt_3"]].isna().mean()

realized_dt_1    0.244306
realized_dt_2    0.438738
realized_dt_3    0.642422
dtype: float64

In [99]:
data_index = data_df.index

In [100]:
def lowercase_first_char(s):
    if len(s) > 0:
        return s[0].lower() + s[1:]
    return s

for i in range(1,4):

    # merge income df
    data_df = data_df.merge(incm_stmt_df, left_on=[data_df.index.get_level_values("symbol"), f"realized_dt_{i}"], right_on=["symbol", "realized_dt"], how="left")\
        .drop(columns=["symbol", "realized_dt"])
        
    data_df.index = data_index

    # rename columns from income df
    data_df.rename(columns={col: "income_stmt_" + lowercase_first_char(col.replace(" ","")) + f"_p{i}y" for col in data_df.columns.intersection(incm_stmt_df.columns)}, inplace=True)

In [101]:
data_df.drop(columns=["realized_dt_1", "realized_dt_2", "realized_dt_3"], inplace=True)

# Remove outliers
In this step, we shall remove symbols with 1 day forward return exceed 100%. This is arbitrary! You can use quantile or percentile to detect outliers here but we'll use the heuristic criteria (100%) for simplicity.

In [102]:
data_df[[f'r{t:02}' for t in T]].describe()

,r01,r05,r10,r21,r42,r63
count,941977.000000,938669.000000,934534.000000,925437.000000,908070.000000,890703.000000
mean,0.005072,0.024830,0.049802,0.106189,0.218177,0.337981
std,4.496026,10.071030,14.274028,20.786351,29.675778,36.697368
min,-0.933718,-0.934104,-0.935632,-0.937143,-0.937407,-0.940266
25%,-0.009434,-0.024096,-0.035714,-0.055046,-0.081420,-0.095694
50%,0.000000,0.000000,0.000000,-0.005263,-0.007519,-0.008130
75%,0.008265,0.019512,0.028169,0.043478,0.066265,0.084615
max,4363.508545,4363.508545,4363.508545,4363.508545,4363.508545,4363.508545


In [103]:
outliers = data_df[data_df["r01"] > 1].index.get_level_values('symbol').unique()
outliers

Index(['AKS', 'JCKH', 'KC', 'KKC', 'NEWS', 'STHAI', 'STOWER'], dtype='object', name='symbol')

In [104]:
data_df = data_df.drop(outliers, level='symbol')

# Create time variables
We could extract time information such as year, month, weekday (i.e. Mon, Tue, Wed, ...) as part of our feature to account for time or seasonality context for a ML model.

In [105]:
data_df['year'] = data_df.index.get_level_values('date').year
data_df['month'] = data_df.index.get_level_values('date').month
data_df['weekday'] = data_df.index.get_level_values('date').weekday

# Store model data

In [107]:
data_df.reset_index().to_csv("data/model_data.csv",index=False)